# The Path to Chess Masterhood

## Question 

What does progress look like for chess players over 2200? 


# Sources 

For chess player ratings for the month of may check out: 
https://ratings.fide.com/download_lists.phtml

In [178]:
import pandas as pd
import matplotlib as plt
from bs4 import BeautifulSoup
import requests
import lxml.html as lh
from requests_html import HTMLSession
from pprint import pprint

In [139]:
# Read in Fide player characteristics (sex, federation, etc)
file = 'players_list_foa_may.txt'
chess = pd.read_fwf(file)

In [177]:
#Filter for chess players  2200 or above 
chess_players_over2200 = chess[chess['SRtng'] >= 2200]

#Exclude Problematic chess players 
chess_players_over2200_clean = chess_players_over2200[(chess_players_over2200["ID Number"] != 11600098) & (chess_players_over2200["ID Number"] != 14108836) & (chess_players_over2200["ID Number"] != 14106329) & (chess_players_over2200["ID Number"] != 929662) & (chess_players_over2200["ID Number"] != 901202)]

# Create list of chess player urls using chess player ids  
all_player_ids = list(chess_players_over2200_clean["ID Number"])
all_player_ids_strings= [str(ids) for ids in all_player_ids]
all_player_urls_prepped = ['https://ratings.fide.com/profile/' + id + '/chart' for id in all_player_ids_strings]
len(all_player_urls_prepped)  


20760

Ignore next block! This is how I checked each chess player's history using rating intervals. 

In [159]:
'''
chess_players_alt = chess[(chess["SRtng"] >= 2200 ) &  (chess["SRtng"] <= 2300 ) ]
chess_players_over2000_clean = chess_players_alt[(chess_players_alt["ID Number"] != 11600098) & (chess_players_alt["ID Number"] != 14108836) & (chess_players_alt["ID Number"] != 14106329) & (chess_players_alt["ID Number"] != 929662) & (chess_players_alt["ID Number"] != 901202)]

all_player_ids = list(chess_players_over2000_clean["ID Number"])
all_player_ids_strings= [str(ids) for ids in all_player_ids]
len(all_player_ids_strings)
'''


12163

In [179]:
# Trim function used to eventually fide ids as the key for the tables so that I can merge history and characteristics 
def trim_url(x):
    y = int(x.replace("https://ratings.fide.com/profile/", "").replace('/chart',""))
    return(y)



The two next blocks were used to store player stats histories in a dictionary. The first block  is a modified version of what we did together (it is a little faster). The second block is how I found specific player pages that could not be scraped (because they were banned, under investigation, or had an error with their id on fide's website). 

In [180]:
player_stats = {}
for link in all_player_urls_prepped: 
    id=trim_url(link)
    page = requests.get(link).text
    dfs = pd.read_html(page, attrs = {'class' : 'profile-table profile-table_chart-table'})
    player_stats[id] = dfs[0]
print(len(player_stats))



20760


In [129]:
'''
player_stats = {}
for link in all_player_urls_prepped: 
    id=trim_url(link)
    page = requests.get(link).text
    try: 
        dfs = pd.read_html(page, attrs = {'class' : 'profile-table profile-table_chart-table'})
    except ValueError:
        print("Error:" + link )

'''




Error:https://ratings.fide.com/profile/901202/chart


In [181]:
def attach_id():
    tags = player_stats.keys()
    for tag in tags: 
        player_stats[tag]["ID Number"] = tag
# Create an ID Number Column for each table and set all values in that column  to the key of the table
    return player_stats
# store each table as variable based on the table id/ table key 
player_stats_with_id = attach_id()


In [182]:
#Turning dictionary into list and then turning list into a massive dataframe
list_of_player_dfs= list(player_stats_with_id.values())
all_player_stats_table= list_of_player_dfs[0].append(list_of_player_dfs[1:-1])


The final merge and filter are supposed to happen in this next block, but I keep getting a memory error. 

In [184]:
#masters_table = chess_players_over2200.merge(all_player_stats_table, on= "ID Number", how= "outer")
#masters_table_filtered = masters_table[masters_table["ID Number"].map(masters_table["ID Number"].value_counts()) > 1]
#len(masters_table_filtered)

MemoryError: Unable to allocate 118. MiB for an array with shape (5, 3093389) and data type float64